# Task 5: Tests

Test your implementation with the following scenarios from the RiMEA guidelines. They provide support for verification and validation of simulation software for crowds. The tests in the guideline may contain features that are not implemented in your cellular automaton. Discuss why you need to implement them to complete the test, or why you can neglect the particular feature and still obtain reasonable test results. A good example for a feature that can be ignored for the tests below is the premovement time (why?). Another one is a (significant) reduction of the
number of pedestrians in RiMEA scenario 4 by shrinking the scenario. This may be necessary if you run into
problems with computational efficiency.

In [ ]:
# always import addroot first
import addroot

In [ ]:
# we will import everything else here
import numpy as np
from src.ca import CrowdModelCellularAutomaton
from src.config import get_save_figure_function
from src.notebook_utils import plot_fundamental_diagrams, plot_average_velocities_per_time_interval
from IPython.display import HTML

save_current_figure = get_save_figure_function("5_rimea_tests")


# set matplotlib to interactive mode -> requires ipympl to be installed
%matplotlib widget

%reload_ext autoreload
%autoreload 2

## 1. TEST1: RiMEA scenario 1 (straight line, ignore premovement time).

Constructing the scenario:
 - $2m \times 40 m$ with a grid unit of $0.4 m$ would result in a $5 \times 100$ cells grid.
 - we can add one extra cell in the length, to compensate for the pedestrian starting in a cell already, ensuring that they must walk the full $40m$.
 - pedestrian should walk at a constant speed of $1.33 \left[\frac{m}{s}\right]$.

In [ ]:
ca1 = CrowdModelCellularAutomaton(grid_size=(5, 101))
ca1.set_cell_pedestrian((3, 0), speed_m_per_s=1.33)
ca1.set_area_target((0, 100), (4, 100))
ca1.save_state()

The test:
 - To test whether the defined speed was actually produced by the simulation, we check the amount of time it takes for all pedestrians (in this case only one) to evacuate the scenario (i.e. be absorbed by a target). The given threshold to pass the test is $[26s, 34s]$.

 - Since movement speed is implemented in a stochastic manner, we can repeat the simulation a couple of times to get a better understanding of the results:

In [ ]:
n_simulations = 100
print(f"Running {n_simulations} simulations...")
evacuation_times = []
passed_tests = []
effective_speeds = []
for _ in range(100):
    ca1.simulate(start_at_saved_state=True, seconds=40, verbose=False)
    evacuation_time = ca1.simulation_total_evacuation_time
    evacuation_times.append(evacuation_time)
    if 26 <= evacuation_time <= 34:
        passed_tests.append(1)
    else:
        passed_tests.append(0)
    effective_speeds.append(40 / evacuation_time)
print("Done!\n")
print(f"Test passed in {100 * sum(passed_tests) / len(passed_tests):.3}% of the runs.")
print(f"Mean +/- standard deviation of time to reach target: "
      f"{np.mean(evacuation_times):.3} +/- {np.std(evacuation_times):.3} s")
print(f"Mean +/- standard deviation of effective speed: {np.mean(effective_speeds):.3} "
      f"+/- {np.std(effective_speeds):.3} m/s")

The test seem to pass around 95% of the time (all the times I checked). This seems to be within the acceptable range of variation, which could just as well occurr naturally e.g. due to measuring errors.  


We can take a look at the interactive visualization of the last simulation to verify that the pedestrian gets absorbed at the reported time.

In [ ]:
print(f"Distance walked: {ca1.get_simulation_average_walked_distance():.3}m")
print(f"Pedestrian reaches target at: {ca1.simulation_total_evacuation_time:.3}s")
print(f"Speed: {ca1.get_simulation_average_speed():.4}m/s")
ca1.plot_simulation_with_time_slider(fig_size=(8, 1.5))

In [ ]:
# once you have selected a slider position that you like:
#save_current_figure("test_1_slider")

## 2. TEST2: RiMEA scenario 4 (fundamental diagram, be careful with periodic boundary conditions).

Constructing the scenario:

 - a corridor $1000 m$ long, $10 m$ wide. This is very large, and can be reasonably simplified by a $100 m$ long corridor.
 - The corridor is to be filled with different densities of persons with an equal as possible free walking speed (for example $1.2$ – $1.4$ m/s). We use $1.3$ here.
 - Densities: $0.5$, $1$, $2$, $3$, $4$, $5$ and $6 \left[\frac{P}{m^2}\right]$.
 - The average speed of pedestrians at the $500m$ mark during $60s$ should be measured. The measurement should start $10s$ after the simulation start.
 
The measuring process is a bit involved, so we simplify it: Instead of measuring at a specific location, we take the average speed of all pedestrians. This should work reasonably well if the speeds of the pedestrians are similar throughout the corridor (no local differences). This holds for basically all the above densities, except for the highest (6), as we will be able to see later.

In [ ]:
def get_model_test_2(density):
    ca = CrowdModelCellularAutomaton(grid_size=(27, 250))
    ca.set_area_obstacle((0, 0), (0, 249))
    ca.set_area_obstacle((26, 0), (26, 249))
    ca.set_area_pedestrian_random_density(density, (1, 0), (25, 248), speed_m_per_s=1.3)
    ca.set_area_target((1, 249), (25, 249))
    ca.save_state()
    return ca

This will run the simulations at each density (takes a while):

In [ ]:
rerun = False  # only set to True if you want to re-run: may take about 40 minutes...

densities = [0.5, 1, 2, 3, 4, 5, 6]

if rerun:
    ca_2_dict = dict()
    average_speed = []
    for density in densities:
        print(f"Running simulation with density of {density} p/m^2")
        ca_2_dict[density] = get_model_test_2(density)
        ca_2_dict[density].simulate(start_at_saved_state=True, seconds=70)
        average_speed.append(ca_2_dict[density].get_simulation_average_speed(from_second=10))
        print("\n")

Now we can take a look at how the average velocities during the simulation changed over time:

In [ ]:
if rerun:
    plot_average_velocities_per_time_interval(ca_2_dict)
    save_current_figure("test_2_speed_per_density")

Clearly, the higher densities (especially 6) are different. In the lower density settings, the speed can be practically considered constant over time. However in the high density setting, that is not the case. We can see this effect in the animation:

In [ ]:
if rerun:
    # will take a while, also maybe 5-15 minutes...
    #HTML(ca_2_dict[6].video_simulation_animation(speed_up=2.0, fig_size=(10, 3)))
    pass

Finally, the resulting fundamental diagrams can be plotted:

In [ ]:
if rerun:
    plot_fundamental_diagrams(densities, average_speed)
    save_current_figure("test_2_fundamental_diagrams")

## 3. TEST3: RiMEA scenario 6 (movement around a corner).

Constructing the scenario:
 - The dimensions of the scenario are given in the diagram of RiMEA figure 5.
 - 20 pedestrians are to be uniformly placed in the starting area.

In [ ]:
ca3 = CrowdModelCellularAutomaton(grid_size=(31, 31))
ca3.set_area_pedestrian_random_n(20, (25, 0), (29, 15))
ca3.set_area_obstacle((24, 0), (24, 24))
ca3.set_area_obstacle((0, 24), (24, 24))
ca3.set_area_obstacle((30, 0), (30, 30))
ca3.set_area_obstacle((0, 30), (30, 30))
ca3.set_area_target((0, 25), (0, 29))
ca3.save_state()

The test:
 - the pedestrians should successfully go around the corner without passing through walls:

In [ ]:
ca3.simulate(start_at_saved_state=True, seconds=45)

In [ ]:
ca3.plot_simulation_end_state()

In [ ]:
ca3.plot_simulation_with_time_slider()

The plots clearly show that the test passes successfully! No walls or corners are passed through.

## 4. TEST4: RiMEA scenario 7 

(demographic parameters, visual comparison of figure and results is sufficient. Simple and correct statistical test gives 5 bonus points if you argue why your choice of test is correct).

Show that a population of 50 adults has a simulation speed distribution dependent on the age, similar to RiMEA figure 2. 

In [ ]:
ca4 = CrowdModelCellularAutomaton(grid_size=(100, 375))
for i in range(0, 100, 2):
    ca4.set_cell_pedestrian((i, 0))
ca4.set_area_target((0, 374), (99, 374))
ca4.save_state()
ca4.plot_state()

In [ ]:
ca4.simulate(start_at_saved_state=True, seconds=60)

In [ ]:
ca4.plot_simulation_end_state()

In [ ]:
ca4.plot_simulation_speed_function_of_age()

In [ ]:
from src.simulation_parameters import SimulationParameters

In [ ]:
sp = SimulationParameters()

In [ ]:
sp.test_distribution_sampling()

Done!